In [1]:
import tensorflow as tf
import os
import sys
from tensorflow import keras
import sklearn
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cv2 import Canny
from skimage.morphology import dilation, erosion, white_tophat, skeletonize
from skimage.morphology import disk 
from skimage import transform
from sklearn.svm import SVC

In [2]:
class_names = ['MALIGNANTMASSES', 'CALC', 'BENIGNMASSES', 'NORM']
class_names_label = {class_names:i for i, class_names in enumerate(class_names)}
print(class_names_label)

IMAGE_SIZE = (224, 224, 3)

def load_data():
    main_directory = r"C:\Users\hp\Documents\CursoML\processed"
    CATEGORY = ["Training", "Testing"]
    
    output = []
    
    for category in CATEGORY: 
        path = os.path.join(main_directory, category)
        print(path)
        
        images = []
        labels = []
        
       # print("Loading {}".format(category))
        
        for folder in os.listdir(path):
            print(folder)
            label = list(class_names_label.values())[int(folder)-1]
            
            for file in os.listdir(os.path.join(path, folder)):
                img_path = os.path.join(os.path.join(path, folder), file)
                
                image = io.imread(img_path, as_gray=True, plugin='matplotlib')
                image = transform.resize(image, IMAGE_SIZE)
                
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')
        
        output.append((images, labels))
        
    return output

{'MALIGNANTMASSES': 0, 'CALC': 1, 'BENIGNMASSES': 2, 'NORM': 3}


In [3]:
(train_images, train_labels), (test_images, test_labels) = load_data() 


C:\Users\hp\Documents\CursoML\processed\Training
1
2
3
4
C:\Users\hp\Documents\CursoML\processed\Testing
1
2
3
4


## Modelo de red neuronal

In [4]:
import keras
#from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import batch_normalization
from keras.layers.advanced_activations import LeakyReLU

#CREAR EL MODELO

cancer_model = tf.keras.models.Sequential() #pila de capas

cancer_model.add(tf.keras.layers.Conv2D(128, kernel_size=(5,5), activation='linear', padding='same', input_shape=(224,224,3))) #capa de convolucion
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D((2, 2),padding='same')) #agrupacion
cancer_model.add(tf.keras.layers.Conv2D(128, kernel_size=(5,5), activation = 'relu'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
cancer_model.add(tf.keras.layers.Conv2D(128, (5, 5), activation='linear', padding='same'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
cancer_model.add(tf.keras.layers.Flatten())
cancer_model.add(tf.keras.layers.Dense(128, activation='linear'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.Dense(4, activation='softmax')) #capa softmax para clasificacion

cancer_model.summary()

cancer_model.compile(optimizer="adam",
                     loss=keras.losses.sparse_categorical_crossentropy,
                     metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 128)     9728      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 128)     409728    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 108, 108, 128)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                    

In [5]:
#Entrenamiento
epochs = 12
batch_size = 128 #numero de muestras por actualizacion 
cancer_train = cancer_model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs)

Epoch 1/12
7/7 [==============================] - 317s 43s/step - loss: 2.4090 - accuracy: 0.4486
Epoch 2/12
7/7 [==============================] - 296s 41s/step - loss: 1.1200 - accuracy: 0.5828
Epoch 3/12
7/7 [==============================] - 292s 41s/step - loss: 0.9313 - accuracy: 0.7170
Epoch 4/12
7/7 [==============================] - 316s 45s/step - loss: 0.7762 - accuracy: 0.7678
Epoch 5/12
7/7 [==============================] - 304s 42s/step - loss: 0.7065 - accuracy: 0.7545
Epoch 6/12
7/7 [==============================] - 302s 42s/step - loss: 0.6872 - accuracy: 0.7860
Epoch 7/12
7/7 [==============================] - 293s 41s/step - loss: 0.6155 - accuracy: 0.8017
Epoch 8/12
7/7 [==============================] - 309s 43s/step - loss: 0.5610 - accuracy: 0.8102
Epoch 9/12
7/7 [==============================] - 294s 42s/step - loss: 0.5556 - accuracy: 0.7896
Epoch 10/12
7/7 [==============================] - 319s 45s/step - loss: 0.5073 - accuracy: 0.8307
Epoch 11/12
7/7 [==

In [9]:
from sklearn.model_selection import StratifiedKFold

folds = 7
skf = StratifiedKFold(n_splits=folds)
skf.get_n_splits(test_images, test_labels)

accuracy_accumulate = 0

for train_index, test_index in skf.split(test_images, test_labels):
    x_test, y_test = test_images[test_index], test_labels[test_index]
    test_eval = cancer_model.evaluate(x_test, y_test)
    accuracy_accumulate += test_eval[1]
    print('Test loss:', test_eval[0])
    print('Test accuracy:', test_eval[1])
    
accuracy_mean = accuracy_accumulate / folds
print("\nAccuracy mean:", accuracy_mean)

2/2 [==============================] - 3s 722ms/step - loss: 1.7256 - accuracy: 0.5111
Test loss: 1.725643277168274
Test accuracy: 0.5111111402511597
2/2 [==============================] - 3s 749ms/step - loss: 0.7606 - accuracy: 0.8444
Test loss: 0.760597288608551
Test accuracy: 0.8444444537162781
2/2 [==============================] - 3s 738ms/step - loss: 0.3137 - accuracy: 0.8667
Test loss: 0.31370460987091064
Test accuracy: 0.8666666746139526
2/2 [==============================] - 3s 759ms/step - loss: 0.2369 - accuracy: 0.9556
Test loss: 0.23692266643047333
Test accuracy: 0.9555555582046509
2/2 [==============================] - 3s 710ms/step - loss: 0.2447 - accuracy: 0.9545
Test loss: 0.24474965035915375
Test accuracy: 0.9545454382896423
2/2 [==============================] - 4s 1s/step - loss: 0.6759 - accuracy: 0.7955
Test loss: 0.6758866906166077
Test accuracy: 0.7954545617103577
2/2 [==============================] - 4s 951ms/step - loss: 0.7182 - accuracy: 0.7955
Test loss

In [115]:
test_eval = cancer_model.evaluate(test_images, test_labels)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

10/10 [==============================] - 69s 5s/step - loss: 0.6928 - accuracy: 0.8301
Test loss: 0.6927545666694641
Test accuracy: 0.8301281929016113


In [121]:
predictions = cancer_model.predict(test_images)
pred_labels = np.argmax(predictions, axis = 1)
print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        96
           1       0.69      0.86      0.77        71
           2       1.00      0.75      0.86        83
           3       0.64      0.74      0.69        62

    accuracy                           0.83       312
   macro avg       0.83      0.82      0.82       312
weighted avg       0.86      0.83      0.84       312

